# Machine Learning in Python - Project 1

Due Friday, March 6th by 5 pm.

## 1. Setup

### 1.1 Libraries

In [1]:
# Add any additional libraries or submodules below

# Display plots inline
%matplotlib inline

# Data libraries
import pandas as pd
import numpy as np

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting defaults
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 80

# sklearn modules
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, KBinsDiscretizer
from sklearn.compose import ColumnTransformer, make_column_transformer
import time

### 1.2 Data

In [2]:
sales = pd.read_csv("sales.csv")
sales_test = pd.read_csv("sales_test.csv")

## 2. Exploratory Data Analysis and Preprocessing

*Include a discussion of the data with a particular emphasis on the features of the data that are relevant for the subsequent modeling. Including visualizations of the data is strongly encouraged - all code and plots must also be described in the write up.*

*In this section you should also implement and describe any preprocessing / transformations of the features. Hint - you should not be modeling this data without transforming some of the features, e.g. modeling sale price directly is not a good idea.*

### 2.1 A glimpse on the dataset.

Looking at a few rows in the dataset.

In [3]:
sales

,sale_price,year_sold,year_built,lot_area,basement_area,living_area,full_bath,half_bath,bedroom,garage_cars,garage_area,ac,zoning,neighborhood,quality,condition
0,244000,2010,1968,11160,2110,2110,2,1,3,2,522,Y,Residential_Low_Density,nb_07,good,average
1,189900,2010,1997,13830,928,1629,2,1,3,2,482,Y,Residential_Low_Density,nb_22,average,average
2,191500,2010,1992,5005,1280,1280,2,0,2,2,506,Y,Residential_Low_Density,nb_10,good,average
3,236500,2010,1995,5389,1595,1616,2,0,2,2,608,Y,Residential_Low_Density,nb_10,good,average
4,189000,2010,1999,7500,994,1804,2,1,3,2,442,Y,Residential_Low_Density,nb_22,good,average
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579,79500,2006,1970,1526,546,1092,1,1,3,0,0,Y,Residential_Medium_Density,nb_02,average,average
1580,160000,2006,1977,17400,1126,1126,2,0,3,2,484,Y,Residential_Low_Density,nb_20,average,average
1581,142500,2006,1984,7937,1003,1003,1,0,3,2,588,Y,Residential_Low_Density,nb_20,average,average
1582,132000,2006,1992,10441,912,970,1,0,3,0,0,Y,Residential_Low_Density,nb_20,average,average


Take a look at the datatypes of the columns.

In [4]:
sales.dtypes

sale_price        int64
year_sold         int64
year_built        int64
lot_area          int64
basement_area     int64
living_area       int64
full_bath         int64
half_bath         int64
bedroom           int64
garage_cars       int64
garage_area       int64
ac               object
zoning           object
neighborhood     object
quality          object
condition        object
dtype: object

Column `ac`, `zoning`, `neighborhood`, `quality`, and `condition` are apparent categorical variables to be encoded. This is also mentioned in the `README.ipynb` file. 

### 2.2 Transform categorical variables using one hot encoding

Here, I would like to discuss how I would transform the categorical variables of this dataset, with rules specified in the `encoding` dictionary.

All the categorical features except `neighborhood` can be easily transformed. However, I decide to sort the `neighborhood` by their means. I will then transform it into an ordinal feature so that this feature can be fitted into a linear model.

In [5]:
rank_neighbor = sales[["sale_price", "neighborhood"]].groupby(['neighborhood'], as_index=False).mean()
rank_neighbor.sort_values("sale_price", inplace=True)
ordering = {
    neighbor: i for i, neighbor in enumerate(rank_neighbor["neighborhood"])
}

If there is air-conditioning (`ac` = `"Y"`), encode it as `1`; otherwise `0`.

Transform `zoning` into an ordinal variable, with `0` being the `Residential_Low_Density` , `1` being the `Residential_Medium_Density` and `2` being the `Residential_High_Density`.

Transform `neighborhood` with the numbering of that neighborhood, e.g., `nb_01` is encoded as `1` and `nb_12` is encoded as `12`.

Transform the `quality` and `condition` columns into ordinals, `poor` as `0`, `fair` as `1`, `good` as `2`, `excellent` as `3`. 

In [6]:
encoding = {
    "ac": {"N": 0, "Y": 1},
    "zoning": {"Residential_Low_Density": 0, "Residential_Medium_Density": 1, "Residential_High_Density": 2},
    "neighborhood": ordering,
    "quality": {"poor": 0, "fair": 1, "average": 2, "good": 3, "excellent": 4},
    "condition": {"poor": 0, "fair": 1, "average": 2, "good": 3, "excellent": 4}
}

Transform the features as specified. I also decide to scale the `sale_price` using a log function and store it in the `log_sale_price` column so it looks more like a normal distribution.

In [7]:
sales.replace(encoding, inplace=True)
sales["log_sale_price"] = np.log(sales["sale_price"])

In [8]:
sales_test.replace(encoding, inplace=True)
y_test = sales_test["sale_price"]
log_y_test = np.log(sales_test["sale_price"])

Feature matrix X of the training data

In [9]:
X = sales.loc[:, "year_sold": "condition"]

Feature matrix X_test of the testing data

In [10]:
X_test = sales_test.loc[:, "year_sold": "condition"]

True label vector y and scaled label vector log_y of the training data

In [11]:
log_y = sales["log_sale_price"]
y = sales["sale_price"]

## 3. Model Fitting and Tuning

The polynomial regression model

In [12]:
degree_poly = 3

poly_model  = make_pipeline(
    make_column_transformer(
        (PolynomialFeatures(degree=1, include_bias=False), ["year_sold"]),
        (PolynomialFeatures(degree=degree_poly, include_bias=False), ["year_built"]),
        (PolynomialFeatures(degree=degree_poly, include_bias=False), ["lot_area"]),
        (PolynomialFeatures(degree=degree_poly, include_bias=False), ["basement_area"]),
        (PolynomialFeatures(degree=degree_poly, include_bias=False), ["living_area"]),
        (PolynomialFeatures(degree=1, include_bias=False), ["full_bath"]), 
        (PolynomialFeatures(degree=1, include_bias=False), ["half_bath"]), 
        (PolynomialFeatures(degree=2, include_bias=False), ["bedroom"]), 
        (PolynomialFeatures(degree=1, include_bias=False), ["garage_cars"]), 
        (PolynomialFeatures(degree=degree_poly, include_bias=False), ["garage_area"]),
        (PolynomialFeatures(degree=1, include_bias=False), ["ac"]),
        (PolynomialFeatures(degree=1, include_bias=False), ["zoning"]),
        (PolynomialFeatures(degree=1, include_bias=False), ["neighborhood"]),
        (PolynomialFeatures(degree=1, include_bias=False), ["quality"]),
        (PolynomialFeatures(degree=1, include_bias=False), ["condition"]),
    ),
    LinearRegression(fit_intercept=True)
)

fit = poly_model.fit(X, log_y)
poly_model.named_steps["columntransformer"].named_transformers_

{'polynomialfeatures-1': PolynomialFeatures(degree=1, include_bias=False, interaction_only=False,
                    order='C'),
 'polynomialfeatures-2': PolynomialFeatures(degree=3, include_bias=False, interaction_only=False,
                    order='C'),
 'polynomialfeatures-3': PolynomialFeatures(degree=3, include_bias=False, interaction_only=False,
                    order='C'),
 'polynomialfeatures-4': PolynomialFeatures(degree=3, include_bias=False, interaction_only=False,
                    order='C'),
 'polynomialfeatures-5': PolynomialFeatures(degree=3, include_bias=False, interaction_only=False,
                    order='C'),
 'polynomialfeatures-6': PolynomialFeatures(degree=1, include_bias=False, interaction_only=False,
                    order='C'),
 'polynomialfeatures-7': PolynomialFeatures(degree=1, include_bias=False, interaction_only=False,
                    order='C'),
 'polynomialfeatures-8': PolynomialFeatures(degree=2, include_bias=False, interaction_only

In [13]:
parameters = {
    'columntransformer__polynomialfeatures-2__degree': np.arange(1, degree_poly + 1),
    'columntransformer__polynomialfeatures-3__degree': np.arange(1, degree_poly + 1),
    'columntransformer__polynomialfeatures-4__degree': np.arange(1, degree_poly + 1),
    'columntransformer__polynomialfeatures-5__degree': np.arange(1, degree_poly + 1),
    'columntransformer__polynomialfeatures-8__degree': np.arange(1, 3),
    'columntransformer__polynomialfeatures-10__degree': np.arange(1, degree_poly + 1),
}

start = time.time()
grid_search = GridSearchCV(poly_model, parameters, 
                           cv=KFold(5, True, random_state=2020), 
                           scoring="neg_root_mean_squared_error", 
                           verbose=1).fit(X, log_y)
print((time.time() - start)/ 60, "minutes")
grid_search

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


5.9478819370269775 minutes


[Parallel(n_jobs=1)]: Done 2430 out of 2430 | elapsed:  5.9min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=2020, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('polynomialfeatures-1',
                                                                         PolynomialFeatures(degree=1,
                                                                                            include_bias=False,
                                                                                            interaction_only=False,
                                                          

In [14]:
print("best index: ", grid_search.best_index_)
print("best param: ", grid_search.best_params_)
print("best score: ", grid_search.best_score_)

best index:  455
best param:  {'columntransformer__polynomialfeatures-10__degree': 3, 'columntransformer__polynomialfeatures-2__degree': 3, 'columntransformer__polynomialfeatures-3__degree': 2, 'columntransformer__polynomialfeatures-4__degree': 1, 'columntransformer__polynomialfeatures-5__degree': 3, 'columntransformer__polynomialfeatures-8__degree': 2}
best score:  -0.11163763369887365


In [15]:
print(len(grid_search.best_estimator_.named_steps['linearregression'].coef_), "Best Coefficients", 
      grid_search.best_estimator_.named_steps['linearregression'].coef_)
print("Best Intercept", grid_search.best_estimator_.named_steps['linearregression'].intercept_)

23 Best Coefficients [-1.21485589e-03  4.00101126e+00 -2.06134021e-03  3.54122877e-07
  7.09056782e-06 -3.58710327e-11  1.59383073e-04  7.20428556e-04
 -1.90450813e-07  2.68390646e-11 -1.63277371e-02  1.09514557e-02
  1.43220645e-02 -8.04598949e-03  2.00172427e-02  4.07605671e-04
 -4.85158920e-07  2.37135186e-10  8.73304598e-02 -3.87181126e-02
  9.44997669e-03  9.85887136e-02  1.18129664e-01]
Best Intercept -2576.8183951853584


In [16]:
log_train_prediction = grid_search.best_estimator_.predict(X)
print("training log rmse", np.sqrt(mean_squared_error(log_y, log_train_prediction )))

training log rmse 0.1085419333345432


In [17]:
print("training rmse", np.sqrt(mean_squared_error(y, np.exp(log_train_prediction ))))

training rmse 20028.210483804847


In [18]:
print("log cross validation rmse", 
      -1 * cross_val_score(grid_search.best_estimator_, X, log_y, 
                           cv=KFold(5, True, random_state=2020), scoring="neg_root_mean_squared_error"))

log cross validation rmse [0.11190339 0.10927243 0.11412519 0.11894092 0.10394623]


In [19]:
log_test_prediction = grid_search.best_estimator_.predict(X_test)
print("testing log rmse", np.sqrt(mean_squared_error(log_y_test, log_test_prediction)))

testing log rmse 0.11856919114776257


In [20]:
print("testing rmse", np.sqrt(mean_squared_error(y_test, np.exp(log_test_prediction))))

testing rmse 22717.48662783491
